In [18]:
import numpy as np
import math
from PIL import Image


class SOM:
    def __init__(self, x_size, y_size, trait_num, t_iter, t_step):
        self.weights = np.random.randint(256, size=(x_size, y_size, trait_num)).astype('float64')
        self.t_iter = t_iter
        self.map_radius = max(self.weights.shape)/2
        self.t_const = self.t_iter/math.log(self.map_radius)
        self.t_step = t_step

    def show(self):
        im = Image.fromarray(self.weights.astype('uint8'), mode='RGB')
        im.format = 'JPG'
        im.show()

    def distance_matrix(self, vector):
        return np.sum((self.weights - vector) ** 2, 2)

    def bmu(self, vector):
        distance = self.distance_matrix(vector)
        mind = distance.argmin()
        result = np.unravel_index(mind, distance.shape)
        return result

    def bmu_distance(self, vector):
        x, y, rgb = self.weights.shape
        xi = np.arange(x).reshape(x, 1).repeat(y, 1)
        
        yi = np.arange(y).reshape(1, y).repeat(x, 0)
        c = np.array(self.bmu(vector))
       
        d = np.dstack((xi, yi)) - c
        print(d)
        result = np.sum(d ** 2, 2)
        #print(result.shape)
        return result

    def hood_radius(self, iteration):
        return self.map_radius * math.exp(-iteration/self.t_const)

    def teach_row(self, vector, i, dis_cut, dist):
        hood_radius_2 = self.hood_radius(i) ** 2
        bmu_distance = self.bmu_distance(vector).astype('float64')
        if dist is None:
            temp = hood_radius_2 - bmu_distance
        else:
            temp = dist ** 2 - bmu_distance
        influence = np.exp(-bmu_distance / (2 * hood_radius_2))
        if dis_cut:
            influence *= ((np.sign(temp) + 1) / 2)
        return np.expand_dims(influence, 2) * (vector - self.weights)

    def teach(self, t_set, distance_cutoff=False, distance=None):
        for i in range(self.t_iter):
            for x in t_set:
                self.weights += self.teach_row(x, i, distance_cutoff, distance)
        self.show()

s = SOM(200, 200, 3, 1, 0.1)
# t_set = np.array([[200, 0, 0], [0, 200, 0], [0, 0, 200], [120, 0, 100]])
t_set = np.random.randint(256, size=(15, 3))
s.teach(t_set)

[[[-62 -87]
  [-62 -86]
  [-62 -85]
  ..., 
  [-62 110]
  [-62 111]
  [-62 112]]

 [[-61 -87]
  [-61 -86]
  [-61 -85]
  ..., 
  [-61 110]
  [-61 111]
  [-61 112]]

 [[-60 -87]
  [-60 -86]
  [-60 -85]
  ..., 
  [-60 110]
  [-60 111]
  [-60 112]]

 ..., 
 [[135 -87]
  [135 -86]
  [135 -85]
  ..., 
  [135 110]
  [135 111]
  [135 112]]

 [[136 -87]
  [136 -86]
  [136 -85]
  ..., 
  [136 110]
  [136 111]
  [136 112]]

 [[137 -87]
  [137 -86]
  [137 -85]
  ..., 
  [137 110]
  [137 111]
  [137 112]]]
[[[-199 -186]
  [-199 -185]
  [-199 -184]
  ..., 
  [-199   11]
  [-199   12]
  [-199   13]]

 [[-198 -186]
  [-198 -185]
  [-198 -184]
  ..., 
  [-198   11]
  [-198   12]
  [-198   13]]

 [[-197 -186]
  [-197 -185]
  [-197 -184]
  ..., 
  [-197   11]
  [-197   12]
  [-197   13]]

 ..., 
 [[  -2 -186]
  [  -2 -185]
  [  -2 -184]
  ..., 
  [  -2   11]
  [  -2   12]
  [  -2   13]]

 [[  -1 -186]
  [  -1 -185]
  [  -1 -184]
  ..., 
  [  -1   11]
  [  -1   12]
  [  -1   13]]

 [[   0 -186]
  [   0 -1